# Operate Cassandra with pyspark 
## Connect to Cassandra

### 下面的bash script是目前確定在Mac上可以執行，讓pyspark在jupyter notebook上運作，並且連到Cassandra。
```
PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS="notebook" ./bin/pyspark --packages com.datastax.spark:spark-cassandra-connector_2.11:2.0.0-M3 --conf spark.cassandra.connection.host=127.0.0.1
```

### 在此之前，試過以下套件：

#### - `cassandra.cluster`：可以分別在python (on jupyter)和pyspark上執行，但暫時無法直接在載入pyspark的jupyter notebook執行。
#### - `pyspark_cassandra`：可以類似`spark-cassandra-connecto`載入，套件的部分改成`--packages TargetHolding:pyspark-cassandra:0.3.5`，但是載入後無法連結Cassandra。

## Load Table From Keyspace

In [2]:
x=sqlContext.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(table="tips", keyspace="ben")\
    .load()

### 觀察資料格式

In [4]:
type(x)

pyspark.sql.dataframe.DataFrame

In [3]:
x.show()

+---+-----+----+------+------+----+------+----------+
|ind|  day| sex|  size|smoker|time|   tip|total_bill|
+---+-----+----+------+------+----+------+----------+
|  5|24.59|3.61|Female|    No| Sun|Dinner|         4|
|100|12.46| 1.5|  Male|    No| Fri|Dinner|         2|
|121|11.69|2.31|  Male|    No|Thur| Lunch|         2|
| 57|38.01| 3.0|  Male|   Yes| Sat|Dinner|         4|
|134|12.26| 2.0|Female|    No|Thur| Lunch|         2|
|165|17.51| 3.0|Female|   Yes| Sun|Dinner|         2|
|108|25.21|4.29|  Male|   Yes| Sat|Dinner|         2|
|208|38.73| 3.0|  Male|   Yes| Sat|Dinner|         4|
| 76|10.51|1.25|  Male|    No| Sat|Dinner|         2|
|127| 8.52|1.48|  Male|    No|Thur| Lunch|         2|
|209|24.27|2.03|  Male|   Yes| Sat|Dinner|         2|
|107|20.49|4.06|  Male|   Yes| Sat|Dinner|         2|
|157|48.17| 5.0|  Male|    No| Sun|Dinner|         6|
|132|20.27|2.83|Female|    No|Thur| Lunch|         2|
| 36|24.06| 3.6|  Male|    No| Sat|Dinner|         3|
| 61|20.29|3.21|  Male|   Ye

### 轉換成Pandas Dataframe

In [5]:
x.toPandas().head()

,ind,day,sex,size,smoker,time,tip,total_bill
0,5,24.59,3.61,Female,No,Sun,Dinner,4
1,100,12.46,1.5,Male,No,Fri,Dinner,2
2,121,11.69,2.31,Male,No,Thur,Lunch,2
3,57,38.01,3.0,Male,Yes,Sat,Dinner,4
4,134,12.26,2.0,Female,No,Thur,Lunch,2


## 各種簡單操作

### 隨機挑選

In [19]:
x.sample(False,0.1).collect()

[Row(ind=u'165', day=u'17.51', sex=u'3.0', size=u'Female', smoker=u'Yes', time=u'Sun', tip=u'Dinner', total_bill=u'2'),
 Row(ind=u'108', day=u'25.21', sex=u'4.29', size=u'Male', smoker=u'Yes', time=u'Sat', tip=u'Dinner', total_bill=u'2'),
 Row(ind=u'107', day=u'20.49', sex=u'4.06', size=u'Male', smoker=u'Yes', time=u'Sat', tip=u'Dinner', total_bill=u'2'),
 Row(ind=u'211', day=u'30.06', sex=u'2.0', size=u'Male', smoker=u'Yes', time=u'Sat', tip=u'Dinner', total_bill=u'3'),
 Row(ind=u'193', day=u'28.44', sex=u'2.56', size=u'Male', smoker=u'Yes', time=u'Thur', tip=u'Lunch', total_bill=u'2'),
 Row(ind=u'33', day=u'15.06', sex=u'3.0', size=u'Female', smoker=u'No', time=u'Sat', tip=u'Dinner', total_bill=u'2'),
 Row(ind=u'114', day=u'23.95', sex=u'2.55', size=u'Male', smoker=u'No', time=u'Sun', tip=u'Dinner', total_bill=u'2'),
 Row(ind=u'106', day=u'15.36', sex=u'1.64', size=u'Male', smoker=u'Yes', time=u'Sat', tip=u'Dinner', total_bill=u'2'),
 Row(ind=u'99', day=u'21.01', sex=u'3.0', size=u'M

### filter（與where一樣）

In [174]:
x.filter("day < 10").collect()

[Row(ind=u'127', day=u'8.52', sex=u'1.48', size=u'Male', smoker=u'No', time=u'Thur', tip=u'Lunch', total_bill=u'2'),
 Row(ind=u'136', day=u'8.51', sex=u'1.25', size=u'Female', smoker=u'No', time=u'Thur', tip=u'Lunch', total_bill=u'2'),
 Row(ind=u'31', day=u'9.55', sex=u'1.45', size=u'Male', smoker=u'No', time=u'Sat', tip=u'Dinner', total_bill=u'2'),
 Row(ind=u'179', day=u'9.6', sex=u'4.0', size=u'Female', smoker=u'Yes', time=u'Sun', tip=u'Dinner', total_bill=u'2'),
 Row(ind=u'146', day=u'8.35', sex=u'1.5', size=u'Female', smoker=u'No', time=u'Thur', tip=u'Lunch', total_bill=u'2'),
 Row(ind=u'7', day=u'8.77', sex=u'2.0', size=u'Male', smoker=u'No', time=u'Sun', tip=u'Dinner', total_bill=u'2'),
 Row(ind=u'68', day=u'3.07', sex=u'1.0', size=u'Female', smoker=u'Yes', time=u'Sat', tip=u'Dinner', total_bill=u'1'),
 Row(ind=u'112', day=u'7.25', sex=u'1.0', size=u'Female', smoker=u'No', time=u'Sat', tip=u'Dinner', total_bill=u'1'),
 Row(ind=u'150', day=u'7.51', sex=u'2.0', size=u'Male', smoker

### 轉成RDD格式

In [13]:
df_x=x.toPandas()
rdd_x=sc.parallelize(df_x)
type(rdd_x)

pyspark.rdd.RDD

### 計算資料列數

In [25]:
x.count()

244

### 改變data type：`withColumn`改變整欄，`cast`則是改變data type。

In [73]:
y=x.withColumn("total_bill", x["total_bill"].cast('float'))

In [74]:
y=y.withColumn("day",y["day"].cast('float'))

In [75]:
y.dtypes

[('ind', 'string'),
 ('day', 'float'),
 ('sex', 'string'),
 ('size', 'string'),
 ('smoker', 'string'),
 ('time', 'string'),
 ('tip', 'string'),
 ('total_bill', 'float')]

### covariance和correlation

In [92]:
y.cov('total_bill','day')

5.0659833456359635

In [76]:
y.corr('total_bill','day')

0.598315121611821

### groupby

In [91]:
y.groupBy('total_bill').avg().collect()

[Row(total_bill=5.0, avg(day)=30.067999649047852, avg(total_bill)=5.0),
 Row(total_bill=2.0, avg(day)=16.448012899129818, avg(total_bill)=2.0),
 Row(total_bill=3.0, avg(day)=23.27763148357994, avg(total_bill)=3.0),
 Row(total_bill=6.0, avg(day)=34.82999897003174, avg(total_bill)=6.0),
 Row(total_bill=1.0, avg(day)=7.242499887943268, avg(total_bill)=1.0),
 Row(total_bill=4.0, avg(day)=28.61351358568346, avg(total_bill)=4.0)]

In [80]:
y.show()

+---+-----+----+------+------+----+------+----------+
|ind|  day| sex|  size|smoker|time|   tip|total_bill|
+---+-----+----+------+------+----+------+----------+
|  5|24.59|3.61|Female|    No| Sun|Dinner|       4.0|
|100|12.46| 1.5|  Male|    No| Fri|Dinner|       2.0|
|121|11.69|2.31|  Male|    No|Thur| Lunch|       2.0|
| 57|38.01| 3.0|  Male|   Yes| Sat|Dinner|       4.0|
|134|12.26| 2.0|Female|    No|Thur| Lunch|       2.0|
|165|17.51| 3.0|Female|   Yes| Sun|Dinner|       2.0|
|108|25.21|4.29|  Male|   Yes| Sat|Dinner|       2.0|
|208|38.73| 3.0|  Male|   Yes| Sat|Dinner|       4.0|
| 76|10.51|1.25|  Male|    No| Sat|Dinner|       2.0|
|127| 8.52|1.48|  Male|    No|Thur| Lunch|       2.0|
|209|24.27|2.03|  Male|   Yes| Sat|Dinner|       2.0|
|107|20.49|4.06|  Male|   Yes| Sat|Dinner|       2.0|
|157|48.17| 5.0|  Male|    No| Sun|Dinner|       6.0|
|132|20.27|2.83|Female|    No|Thur| Lunch|       2.0|
| 36|24.06| 3.6|  Male|    No| Sat|Dinner|       3.0|
| 61|20.29|3.21|  Male|   Ye

### Repartition

In [184]:
y.repartition("total_bill").show()

+---+-----+----+------+------+----+------+----------+
|ind|  day| sex|  size|smoker|time|   tip|total_bill|
+---+-----+----+------+------+----+------+----------+
|186|20.69| 5.0|  Male|    No| Sun|Dinner|       5.0|
|156|29.85|5.14|Female|    No| Sun|Dinner|       5.0|
|217|28.15| 3.0|  Male|   Yes| Sat|Dinner|       5.0|
|143|41.19| 5.0|  Male|    No|Thur| Lunch|       5.0|
|188|30.46| 2.0|  Male|   Yes| Sun|Dinner|       5.0|
|100|12.46| 1.5|  Male|    No| Fri|Dinner|       2.0|
|121|11.69|2.31|  Male|    No|Thur| Lunch|       2.0|
|134|12.26| 2.0|Female|    No|Thur| Lunch|       2.0|
|165|17.51| 3.0|Female|   Yes| Sun|Dinner|       2.0|
|108|25.21|4.29|  Male|   Yes| Sat|Dinner|       2.0|
| 76|10.51|1.25|  Male|    No| Sat|Dinner|       2.0|
|127| 8.52|1.48|  Male|    No|Thur| Lunch|       2.0|
|209|24.27|2.03|  Male|   Yes| Sat|Dinner|       2.0|
|107|20.49|4.06|  Male|   Yes| Sat|Dinner|       2.0|
|132|20.27|2.83|Female|    No|Thur| Lunch|       2.0|
| 61|20.29|3.21|  Male|   Ye

### Replace elements

In [131]:
y.replace("No","Ng").show()

+---+-----+----+------+------+----+------+----------+
|ind|  day| sex|  size|smoker|time|   tip|total_bill|
+---+-----+----+------+------+----+------+----------+
|  5|24.59|3.61|Female|    Ng| Sun|Dinner|       4.0|
|100|12.46| 1.5|  Male|    Ng| Fri|Dinner|       2.0|
|121|11.69|2.31|  Male|    Ng|Thur| Lunch|       2.0|
| 57|38.01| 3.0|  Male|   Yes| Sat|Dinner|       4.0|
|134|12.26| 2.0|Female|    Ng|Thur| Lunch|       2.0|
|165|17.51| 3.0|Female|   Yes| Sun|Dinner|       2.0|
|108|25.21|4.29|  Male|   Yes| Sat|Dinner|       2.0|
|208|38.73| 3.0|  Male|   Yes| Sat|Dinner|       4.0|
| 76|10.51|1.25|  Male|    Ng| Sat|Dinner|       2.0|
|127| 8.52|1.48|  Male|    Ng|Thur| Lunch|       2.0|
|209|24.27|2.03|  Male|   Yes| Sat|Dinner|       2.0|
|107|20.49|4.06|  Male|   Yes| Sat|Dinner|       2.0|
|157|48.17| 5.0|  Male|    Ng| Sun|Dinner|       6.0|
|132|20.27|2.83|Female|    Ng|Thur| Lunch|       2.0|
| 36|24.06| 3.6|  Male|    Ng| Sat|Dinner|       3.0|
| 61|20.29|3.21|  Male|   Ye

### 針對欄位取代元素

In [130]:
y.withColumn('sex',y.sex).replace('3.0','test').show()

+---+-----+----+------+------+----+------+----------+
|ind|  day| sex|  size|smoker|time|   tip|total_bill|
+---+-----+----+------+------+----+------+----------+
|  5|24.59|3.61|Female|    No| Sun|Dinner|       4.0|
|100|12.46| 1.5|  Male|    No| Fri|Dinner|       2.0|
|121|11.69|2.31|  Male|    No|Thur| Lunch|       2.0|
| 57|38.01|test|  Male|   Yes| Sat|Dinner|       4.0|
|134|12.26| 2.0|Female|    No|Thur| Lunch|       2.0|
|165|17.51|test|Female|   Yes| Sun|Dinner|       2.0|
|108|25.21|4.29|  Male|   Yes| Sat|Dinner|       2.0|
|208|38.73|test|  Male|   Yes| Sat|Dinner|       4.0|
| 76|10.51|1.25|  Male|    No| Sat|Dinner|       2.0|
|127| 8.52|1.48|  Male|    No|Thur| Lunch|       2.0|
|209|24.27|2.03|  Male|   Yes| Sat|Dinner|       2.0|
|107|20.49|4.06|  Male|   Yes| Sat|Dinner|       2.0|
|157|48.17| 5.0|  Male|    No| Sun|Dinner|       6.0|
|132|20.27|2.83|Female|    No|Thur| Lunch|       2.0|
| 36|24.06| 3.6|  Male|    No| Sat|Dinner|       3.0|
| 61|20.29|3.21|  Male|   Ye

### `rollup`跟groupby很像，不過會多一列總數的計算

In [140]:
y.groupBy("size","total_bill").count().show()

+------+----------+-----+
|  size|total_bill|count|
+------+----------+-----+
|  Male|       2.0|   98|
|  Male|       3.0|   24|
|  Male|       4.0|   28|
|Female|       4.0|    9|
|  Male|       1.0|    1|
|Female|       1.0|    3|
|  Male|       5.0|    4|
|Female|       5.0|    1|
|Female|       6.0|    2|
|Female|       2.0|   58|
|  Male|       6.0|    2|
|Female|       3.0|   14|
+------+----------+-----+



In [141]:
y.rollup("size","total_bill").count().show()

+------+----------+-----+
|  size|total_bill|count|
+------+----------+-----+
|  Male|      null|  157|
|  Male|       4.0|   28|
|  Male|       3.0|   24|
|  null|      null|  244|
|  Male|       5.0|    4|
|  Male|       2.0|   98|
|Female|       6.0|    2|
|Female|       3.0|   14|
|Female|       4.0|    9|
|Female|       2.0|   58|
|  Male|       1.0|    1|
|Female|       1.0|    3|
|  Male|       6.0|    2|
|Female|       5.0|    1|
|Female|      null|   87|
+------+----------+-----+



### Select

In [144]:
y.select('sex').show()

+----+
| sex|
+----+
|3.61|
| 1.5|
|2.31|
| 3.0|
| 2.0|
| 3.0|
|4.29|
| 3.0|
|1.25|
|1.48|
|2.03|
|4.06|
| 5.0|
|2.83|
| 3.6|
|3.21|
| 2.0|
| 3.0|
|4.08|
|2.75|
+----+
only showing top 20 rows



In [147]:
y.selectExpr('total_bill*100').show()

+------------------+
|(total_bill * 100)|
+------------------+
|             400.0|
|             200.0|
|             200.0|
|             400.0|
|             200.0|
|             200.0|
|             200.0|
|             400.0|
|             200.0|
|             200.0|
|             200.0|
|             200.0|
|             600.0|
|             200.0|
|             300.0|
|             200.0|
|             300.0|
|             200.0|
|             200.0|
|             200.0|
+------------------+
only showing top 20 rows



### 排序（sort和orderBy）

In [186]:
y.sort('total_bill',ascending=False).show()

+---+-----+----+------+------+----+------+----------+
|ind|  day| sex|  size|smoker|time|   tip|total_bill|
+---+-----+----+------+------+----+------+----------+
|142| 34.3| 6.7|  Male|    No|Thur| Lunch|       6.0|
|126| 29.8| 4.2|Female|    No|Thur| Lunch|       6.0|
|157|48.17| 5.0|  Male|    No| Sun|Dinner|       6.0|
|144|27.05| 5.0|Female|    No|Thur| Lunch|       6.0|
|217|28.15| 3.0|  Male|   Yes| Sat|Dinner|       5.0|
|188|30.46| 2.0|  Male|   Yes| Sun|Dinner|       5.0|
|186|20.69| 5.0|  Male|    No| Sun|Dinner|       5.0|
|143|41.19| 5.0|  Male|    No|Thur| Lunch|       5.0|
|156|29.85|5.14|Female|    No| Sun|Dinner|       5.0|
|205|20.53| 4.0|  Male|   Yes|Thur| Lunch|       4.0|
| 24|39.42|7.58|  Male|    No| Sat|Dinner|       4.0|
|220|30.14|3.09|Female|   Yes| Sat|Dinner|       4.0|
| 53|34.81| 5.2|Female|    No| Sun|Dinner|       4.0|
| 96|40.17|4.73|  Male|   Yes| Fri|Dinner|       4.0|
|161| 21.5| 3.5|  Male|    No| Sun|Dinner|       4.0|
|208|38.73| 3.0|  Male|   Ye